# Binary Search Benchmark Injection Testing

## Power Trace Gathering

At this point you've got to insert code to perform the power trace capture. There are two options here:
* Capture from physical device.
* Read from a file.

You get to choose your adventure - see the two notebooks with the same name of this, but called `(SIMULATED)` or `(HARDWARE)` to continue. Inside those notebooks you should get some code to copy into the following section, which will define the capture function.

Be sure you get the `"✔️ OK to continue!"` print once you run the next cell, otherwise things will fail later on!


In [167]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEXMEGA'
%run "../../Setup_Scripts/Setup_Generic.ipynb"
scope.adc.samples = 800

INFO: Found ChipWhisperer😍


In [ ]:
scope.dis()
target.dis()

In [157]:
#import necessary library
import linear_comp as lc
import matplotlib.pyplot as plt 
import chipwhisperer as cw
import numpy as np

In [158]:
def cap_pass_trace():
    scope.arm()
    ret = scope.capture()
    num_char = target.in_waiting()
    while num_char > 0:
        target.read(num_char, 10)
        time.sleep(0.01)
        num_char = target.in_waiting()

    trace = scope.get_last_trace()
    return trace

<h3> This is for Research Benchmark Checking </h3>

In [163]:
%%bash -s "$PLATFORM" "$SS_VER"
cd ../../hardware/victims/firmware/Bi_search
gmake PLATFORM=$1 CRYPTO_TARGET=NONE SS_VER=$2 -j

SS_VER set to SS_VER_1_1
SS_VER set to SS_VER_1_1
SS_VER set to SS_VER_1_1
SS_VER set to SS_VER_1_1
gmake[1]: '.dep' is up to date.
SS_VER set to SS_VER_1_1
SS_VER set to SS_VER_1_1
.
Welcome to another exciting ChipWhisperer target build!!
avr-gcc (Homebrew AVR GCC 9.4.0) 9.4.0
Copyright (C) 2019 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

.
Compiling:
-en     .././hal/xmega/XMEGA_AES_driver.c ...
-e Done!
.
Compiling:
-en     .././hal/xmega/uart.c ...
-e Done!
.
Compiling:
-en     .././hal/xmega/xmega_hal.c ...
-e Done!
.
Compiling:
-en     bi_search.c ...
-e Done!
.
Compiling:
-en     .././hal/xmega/usart_driver.c ...
-e Done!
.
LINKING:
-en     bi_search-CWLITEXMEGA.elf ...
-e Done!
.
Creating load file for Flash: bi_search-CWLITEXMEGA.hex
avr-objcopy -O ihex -R .eeprom -R .fuse -R .lock -R .signature bi_search-CWLITEXMEGA.elf bi_search-CWLITEXM

In [164]:
cw.program_target(scope, prog, "../../hardware/victims/firmware/Bi_search/bi_search-{}.hex".format(PLATFORM))

XMEGA Programming flash...
XMEGA Reading flash...
Verified flash OK, 2017 bytes


In [89]:
#reset_target(scope)
trace_comp = cap_pass_trace()
#cw.plot(trace_comp)


In [161]:
reset_target(scope)
trace_set=[]
for i in range(150):
    trace_hit = cap_pass_trace()
    if(len(trace_set))==0:
        trace_set.append(trace_hit)
    else:
        flag=False
        for j in range(len(trace_set)):
            if (lc.linear_comp(trace_hit,trace_set[j],0.1,250))>750:
                flag=True
        
        if not flag:
            trace_set.append(trace_hit)
    
print(len(trace_set))

2


In [ ]:
#cw.plot(trace_comp) * cw.plot(trace_set[0])
#print(lc.linear_comp(trace_comp,trace_set[0],0.1,250))
#abs(np.sum(trace_set[1]-trace_comp))

cw.plot(trace_set[0]) *  cw.plot(trace_set[1])
#abs(np.sum(trace_set[0]-trace_set[2]))
#print(lc.linear_comp(trace_set[0],trace_set[2],0.1,250))

# The Testing Part

<h3> Check False Positives </h3>

 - Iterate over a thousand times and the result should be 1000

<h3> Check Accruacy </h3>

 - Iterate over 1024 times and there should be only 4 ROP attacks

In [165]:
result=[]
reset_target(scope)

for i in range(1024): # we now are getting right result
    trace_test=cap_pass_trace()
    x=max([lc.linear_comp(trace_test,trace_set[j],0.1,250) for j in range(len(trace_set))])
    result.append(x)

In [166]:
with open("bi_search_ROP_accuracy.csv",'w') as f:
    f.write("Test Iteration,Matching,Decision\n")
    for i in range(len(result)):
        if(result[i]>750):
            f.write(f"{i},{result[i]},OK\n")
        else:
            f.write(f"{i},{result[i]},Failed\n")